In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    NewsURLLoader,
    TrelloLoader,
)
from unstructured.cleaners.core import clean_extra_whitespace
from langchain.text_splitter import TokenTextSplitter
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
TRELLO_API_KEY= os.getenv('TRELLO_API_KEY')
TRELLO_TOKEN = os.getenv('TRELLO_TOKEN')

In [4]:

# # Get the open cards from "Awesome Board"
# loader = TrelloLoader.from_credentials(
#     "SOW data",
#     api_key=TRELLO_API_KEY,
#     token=TRELLO_TOKEN,
#     card_filter="open",
# )
# documents = loader.load()

# print(documents[0].page_content)
# print(documents[0].metadata)

In [5]:
# # Get all the cards from "Awesome Board" but only include the
# # card list(column) as extra metadata.
# loader = TrelloLoader.from_credentials(
#     "SOW data",
#     api_key=TRELLO_API_KEY,
#     token=TRELLO_TOKEN,
#     extra_metadata=("list"),
# )
# documents = loader.load()

# print(documents[0].page_content)
# print(documents[0].metadata)

In [6]:
# documents

In [7]:
client_id=os.getenv('TODO_CLIENT_ID')
token=os.getenv('TODO_TOKEN')
BASE_URL = 'https://graph.microsoft.com/v1.0/me/todo/lists'

In [8]:
# Oauth settings
import os
import pickle
import time

import yaml
from requests_oauthlib import OAuth2Session

settings = {
    "redirect": "https://localhost/login/authorized",
    "scopes": "openid offline_access tasks.readwrite",
    "authority": "https://login.microsoftonline.com/common",
    "authorize_endpoint": "/oauth2/v2.0/authorize",
    "token_endpoint": "/oauth2/v2.0/token",
}

# Code taken from https://docs.microsoft.com/en-us/graph/tutorials/python?tutorial-step=3

# This is necessary because Azure does not guarantee
# to return scopes in the same case and order as requested
os.environ["OAUTHLIB_RELAX_TOKEN_SCOPE"] = "1"
os.environ["OAUTHLIB_IGNORE_SCOPE_CHANGE"] = "1"

redirect = settings["redirect"]
scope = settings["scopes"]

authorize_url = "{0}{1}".format(settings["authority"], settings["authorize_endpoint"])
token_url = "{0}{1}".format(settings["authority"], settings["token_endpoint"])

# User settings location
config_dir = "{}/.config/tod0".format(os.path.expanduser("~"))
if not os.path.isdir(config_dir):
    os.makedirs(config_dir)


def check_keys(keys):
    client_id = keys["client_id"]
    client_secret = keys["client_secret"]

    if client_id == "" or client_secret == "":
        print(
            "Please enter your client id and secret in {}".format(
                os.path.join(config_dir, "keys.yml")
            )
        )
        print(
            "Instructions to getting your API client id and secret can be found here:\n{}".format(
                "https://github.com/kiblee/tod0/blob/master/GET_KEY.md"
            )
        )
        exit()


# Check for api keys
keys_path = os.path.join(config_dir, "keys.yml")
if not os.path.isfile(keys_path):
    keys = {"client_id": "", "client_secret": ""}

    with open(keys_path, "w") as f:
        yaml.dump(keys, f)
    check_keys(keys)
else:
    # Load api keys
    with open(keys_path) as f:
        keys = yaml.load(f, yaml.SafeLoader)
        check_keys(keys)

client_id = keys["client_id"]
client_secret = keys["client_secret"]


def get_token():
    try:
        # Try to load token from local
        with open(os.path.join(config_dir, "token.pkl"), "rb") as f:
            token = pickle.load(f)

        token = refresh_token(token)

    except Exception:
        # Authorize user to get token
        outlook = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)

        # Redirect  the user owner to the OAuth provider
        authorization_url, state = outlook.authorization_url(authorize_url)
        print("Please go here and authorize:\n", authorization_url)

        # Get the authorization verifier code from the callback url
        redirect_response = input("Paste the full redirect URL below:\n")

        # Fetch the access token
        token = outlook.fetch_token(
            token_url,
            client_secret=client_secret,
            authorization_response=redirect_response,
        )

    store_token(token)
    return token


def store_token(token):
    with open(os.path.join(config_dir, "token.pkl"), "wb") as f:
        pickle.dump(token, f)


def refresh_token(token):
    # Check expiration
    now = time.time()
    # Subtract 5 minutes from expiration to account for clock skew
    expire_time = token["expires_at"] - 300
    if now >= expire_time:
        # Refresh the token
        aad_auth = OAuth2Session(
            client_id, token=token, scope=scope, redirect_uri=redirect
        )

        refresh_params = {"client_id": client_id, "client_secret": client_secret}

        new_token = aad_auth.refresh_token(token_url, **refresh_params)
        return new_token

    # Token still valid, just return it
    return token


def get_oauth_session():
    token = get_token()
    return OAuth2Session(client_id, scope=scope, token=token)

In [9]:
settings = {
    "redirect": "https://localhost/login/authorized",
    "scopes": "openid offline_access tasks.readwrite",
    "authority": "https://login.microsoftonline.com/common",
    "authorize_endpoint": "/oauth2/v2.0/authorize",
    "token_endpoint": "/oauth2/v2.0/token",
}

In [10]:
redirect = settings["redirect"]
scope = settings["scopes"]

In [11]:
import json
from requests_oauthlib import OAuth2Session

In [12]:
"""
For implementation details, refer to this source:
https://docs.microsoft.com/en-us/graph/api/resources/todo-overview?view=graph-rest-1.0
"""
import json
from datetime import datetime
from typing import Union

from todocli.models.todolist import TodoList
from todocli.models.todotask import Task, TaskStatus
from todocli.graphapi.oauth import get_oauth_session

from todocli.utils.datetime_util import datetime_to_api_timestamp

BASE_URL = "https://graph.microsoft.com/v1.0/me/todo/lists"


class ListNotFound(Exception):
    def __init__(self, list_name):
        self.message = "List with name '{}' could not be found".format(list_name)
        super(ListNotFound, self).__init__(self.message)


class TaskNotFoundByName(Exception):
    def __init__(self, task_name, list_name):
        self.message = "Task with name '{}' could not be found in list '{}'".format(
            task_name, list_name
        )
        super(TaskNotFoundByName, self).__init__(self.message)


class TaskNotFoundByIndex(Exception):
    def __init__(self, task_index, list_name):
        self.message = "Task with index '{}' could not be found in list '{}'".format(
            task_index, list_name
        )
        super(TaskNotFoundByIndex, self).__init__(self.message)


def parse_response(response):
    return json.loads(response.content.decode())["value"]


def get_lists():
    session = get_oauth_session()
    response = session.get(BASE_URL)
    response_value = parse_response(response)
    return response_value



def create_list(title: str):
    request_body = {"displayName": title}
    session = get_oauth_session()
    response = session.post(BASE_URL, json=request_body)
    return True if response.ok else response.raise_for_status()


# TODO No associated command
def rename_list(old_title: str, new_title: str):
    list_id = get_list_id_by_name(old_title)
    request_body = {"title": new_title}
    session = get_oauth_session()
    response = session.patch(f"{BASE_URL}/{list_id}", json=request_body)
    return True if response.ok else response.raise_for_status()


def get_tasks(list_name: str = None, list_id: str = None, num_tasks: int = 100):
    assert (list_name is not None) or (
        list_id is not None
    ), "You must provide list_name or list_id"

    # For compatibility with cli
    if list_id is None:
        list_id = get_list_id_by_name(list_name)

    endpoint = (
        f"{BASE_URL}/{list_id}/tasks?$filter=status ne 'completed'&$top={num_tasks}"
    )
    session = get_oauth_session()
    response = session.get(endpoint)
    response_value = parse_response(response)
    return response_value



def create_task(
    task_name: str,
    list_name: str = None,
    list_id: str = None,
    reminder_datetime: datetime = None,
):
    assert (list_name is not None) or (
        list_id is not None
    ), "You must provide list_name or list_id"

    # For compatibility with cli
    if list_id is None:
        list_id = get_list_id_by_name(list_name)

    endpoint = f"{BASE_URL}/{list_id}/tasks"
    request_body = {
        "title": task_name,
        "reminderDateTime": datetime_to_api_timestamp(reminder_datetime),
    }
    session = get_oauth_session()
    response = session.post(endpoint, json=request_body)
    return True if response.ok else response.raise_for_status()


def complete_task(
    list_name: str = None,
    task_name: Union[str, int] = None,
    list_id: str = None,
    task_id: str = None,
):
    assert (list_name is not None) or (
        list_id is not None
    ), "You must provide list_name or list_id"
    assert (task_name is not None) or (
        task_id is not None
    ), "You must provide task_name or task_id"

    # For compatibility with cli
    if list_id is None:
        list_id = get_list_id_by_name(list_name)
    if task_id is None:
        task_id = get_task_id_by_name(list_name, task_name)

    endpoint = f"{BASE_URL}/{list_id}/tasks/{task_id}"
    request_body = {
        "status": TaskStatus.COMPLETED,
        "completedDateTime": datetime_to_api_timestamp(datetime.now()),
    }
    session = get_oauth_session()
    response = session.patch(endpoint, json=request_body)
    return True if response.ok else response.raise_for_status()


def remove_task(list_name: str, task_name: Union[str, int]):
    list_id = get_list_id_by_name(list_name)
    task_id = get_task_id_by_name(list_name, task_name)
    endpoint = f"{BASE_URL}/{list_id}/tasks/{task_id}"
    session = get_oauth_session()
    response = session.delete(endpoint)
    return True if response.ok else response.raise_for_status()


def get_list_id_by_name(list_name):
    endpoint = f"{BASE_URL}?$filter=startswith(displayName,'{list_name}')"
    session = get_oauth_session()
    response = session.get(endpoint)
    response_value = parse_response(response)
    try:
        return response_value[0]["id"]
    except IndexError:
        raise ListNotFound(list_name)


def get_task_id_by_name(list_name: str, task_name: str):
    if isinstance(task_name, str):
        try:
            list_id = get_list_id_by_name(list_name)
            endpoint = f"{BASE_URL}/{list_id}/tasks?$filter=title eq '{task_name}'"
            session = get_oauth_session()
            response = session.get(endpoint)
            response_value = parse_response(response)
            return [Task(x) for x in response_value][0].id
        except IndexError:
            raise TaskNotFoundByName(task_name, list_name)
    # elif isinstance(task_name, int):
    #    tasks = get_tasks(list_name, task_list_position + 1)
    #    try:
    #        return tasks[task_list_position].id
    #    except IndexError:
    #        raise TaskNotFoundByIndex(task_list_position, list_name)
    else:
        raise

In [15]:
response_value = get_tasks(list_name='Tasks')
response_value[0:2]

[{'@odata.etag': 'W/"wyYSbsYOaUCZPsQATrx1BgAE5CArjA=="',
  'importance': 'normal',
  'isReminderOn': False,
  'status': 'notStarted',
  'title': 'Read Book',
  'createdDateTime': '2024-01-12T01:43:03.9453132Z',
  'lastModifiedDateTime': '2024-01-12T01:45:45.6163949Z',
  'hasAttachments': False,
  'categories': [],
  'id': 'AQMkADAwATMwMAItMTg4AGMtZmRiNC0wMAItMDAKAEYAAAORlXbjKGFLSJiBLf4_KVkjBwDDJhJuxg5pQJk_xABOvHUGAAACARIAAADDJhJuxg5pQJk_xABOvHUGAATj9f_tAAAA',
  'body': {'content': '', 'contentType': 'text'}},
 {'@odata.etag': 'W/"wyYSbsYOaUCZPsQATrx1BgAE5CArUw=="',
  'importance': 'normal',
  'isReminderOn': False,
  'status': 'notStarted',
  'title': 'setup ocr on server',
  'createdDateTime': '2024-01-11T02:06:42.9317049Z',
  'lastModifiedDateTime': '2024-01-12T01:42:20.4559906Z',
  'hasAttachments': False,
  'categories': [],
  'id': 'AQMkADAwATMwMAItMTg4AGMtZmRiNC0wMAItMDAKAEYAAAORlXbjKGFLSJiBLf4_KVkjBwDDJhJuxg5pQJk_xABOvHUGAAACARIAAADDJhJuxg5pQJk_xABOvHUGAATj9f_lAAAA',
  'body': {

In [17]:
response_value = get_lists()
response_value[0:2]

[{'@odata.etag': 'W/"wyYSbsYOaUCZPsQATrx1BgAE5B9mMQ=="',
  'displayName': 'Tasks',
  'isOwner': True,
  'isShared': False,
  'wellknownListName': 'defaultList',
  'id': 'AQMkADAwATMwMAItMTg4AGMtZmRiNC0wMAItMDAKAC4AAAORlXbjKGFLSJiBLf4_KVkjAQDDJhJuxg5pQJk_xABOvHUGAAACARIAAAA='},
 {'@odata.etag': 'W/"wyYSbsYOaUCZPsQATrx1BgAE4T+1bQ=="',
  'displayName': '2022',
  'isOwner': True,
  'isShared': False,
  'wellknownListName': 'none',
  'id': 'AQMkADAwATMwMAItMTg4AGMtZmRiNC0wMAItMDAKAC4AAAORlXbjKGFLSJiBLf4_KVkjAQDDJhJuxg5pQJk_xABOvHUGAAMPg-liAAAA'}]

In [ ]:
# create_task(task_name='hello', list_name='Tasks', reminder_datetime=datetime.now())

True